## Download and extract dataset From [kaggle.com](https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020)

In [1]:
#!kaggle datasets download -d hernan4444/anime-recommendation-database-2020
#!unzip -o -j anime-recommendation-database-2020.zip

## Import library dan load dataset

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
data = pd.read_csv("anime_with_synopsis.csv")
data.head(5)

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


## Handling Missing Value

In [4]:
# Check for missing values
missing_values = data.isnull().sum()
missing_values

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    8
dtype: int64

In [5]:
data = data.dropna(how='any')\
    .reset_index(drop=True)\
    .copy()
missing_values = data.isnull().sum()
missing_values

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
16201,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...
16202,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...
16203,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .
16204,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.


## feature extraction with TF-IDF

In [7]:
tfidf = TfidfVectorizer()

tfidf.fit(data['Genres'].values.astype('U'))#U mean Unicode

tfidf_matrix = tfidf.transform(data['Genres'].values.astype('U'))

In [8]:
# Creating Cosine Similarity Matrix
cos_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cos_sim_matrix.shape)

(16206, 16206)


## Content Based Recomender

In [9]:
def recommend_anime(anime_title, cosine_sim_matrix, df, top_n=5):
    # Get the index of the anime title
    idx = data[data['Name'] == anime_title].index[0]
    sim_scores = list(enumerate(cos_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_anime = [i[0] for i in sim_scores[1:top_n]]
    recommended_anime = data.loc[top_anime, ['Name', 'Genres']]

    return recommended_anime

## Trying Content Based Recomender

In [10]:
anime_title = input("Anime Title : ")
top_n = 5

recommended_anime = recommend_anime(anime_title, cos_sim_matrix, data, top_n)
recommended_anime.to_csv('recommended_anime.csv', index=False)
recommended_anime_csv = pd.read_csv('recommended_anime.csv')
recommended_anime_csv

,Name,Genres
0,Paprika,"Dementia, Fantasy, Horror, Mystery, Psychologi..."
1,Higurashi no Naku Koro ni,"Mystery, Dementia, Horror, Psychological, Supe..."
2,Higurashi no Naku Koro ni Gou,"Mystery, Dementia, Horror, Psychological, Supe..."
3,Higurashi no Naku Koro ni Sotsu,"Mystery, Dementia, Horror, Psychological, Supe..."
4,Yakusoku no Neverland,"Sci-Fi, Mystery, Horror, Psychological, Thrill..."
